In [173]:
import pandas as pd
import sys
import os

In [174]:
DEBUG = False

# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../") if DEBUG else os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [175]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [176]:
# Leitura do arquivo
df_silver = extrair_dados_database("alunos/alunos.parquet", "silver")

# Carregando as dimensoes
df_d_aluno = extrair_dados_database("d_aluno.parquet", "gold")
df_d_periodo = extrair_dados_database("d_periodo.parquet", "gold")
df_d_curso = extrair_dados_database("d_curso.parquet", "gold")

# CR

In [177]:
df_cr = df_silver.dropna(subset='crPorPeriodo').copy()

# Converte a coluna 'crPorPeriodo' para uma lista de tuplas
df_cr["crPorPeriodo"] = df_cr["crPorPeriodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
# Explode a coluna 'crPorPeriodo' para dividir as listas em linhas separadas
df_cr_exploded = df_cr.explode("crPorPeriodo")

# Cria um novo DataFrame com as tuplas da coluna 'crPorPeriodo' separadas em colunas 'periodo' e 'crPorPeriodo'
df_crPeriodo = pd.DataFrame(
    df_cr_exploded["crPorPeriodo"].tolist(),
    index=df_cr_exploded.index,
    columns=("periodo", "cr_periodo"),
)

# Adiciona as colunas 'matriculaDre' e 'periodo_temp' ao novo DataFrame
df_crPeriodo["matriculaDre"] = df_cr_exploded["matriculaDre"].values
df_crPeriodo.dropna(inplace=True)
df_crPeriodo.drop_duplicates(inplace=True)

# CRA

In [178]:
df_cra = df_silver.dropna(subset='craPorPeriodo').copy()

# Converte a coluna 'crPorPeriodo' para uma lista de tuplas
df_cra["craPorPeriodo"] = df_cra["craPorPeriodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
# Explode a coluna 'crPorPeriodo' para dividir as listas em linhas separadas
df_cra_exploded = df_cra.explode("craPorPeriodo")

# Cria um novo DataFrame com as tuplas da coluna 'crPorPeriodo' separadas em colunas 'periodo' e 'cr_periodo'
df_craPeriodo = pd.DataFrame(
    df_cra_exploded["craPorPeriodo"].tolist(),
    index=df_cra_exploded.index,
    columns=("periodo", "cr_acumulado"),
)
# Adiciona a coluna 'matriculaDre' ao DataFrame 'df_craPeriodo'
df_craPeriodo["matriculaDre"] = df_cra_exploded["matriculaDre"].values
df_craPeriodo.dropna(inplace=True)
df_craPeriodo.drop_duplicates(inplace=True)
# df_craPeriodo['cr_acumulado'] = df_craPeriodo['cr_acumulado'].fillna(value=0)

# MERGER

In [179]:
df_silver = df_silver[['matriculaDre', 'codCursoIngresso', 'codCursoAtual']]
df_crs = df_craPeriodo.merge(df_crPeriodo, how='outer')
df_situacao_periodo = df_silver.merge(df_crs)

In [180]:
# Padronizando nome das colunas
f_situacao_periodo = df_situacao_periodo.rename(
    columns={
        "matriculaDre": "DS_MATRICULA_DRE",
        "periodo": "DS_PERIODO",
        "cr_acumulado": "VL_CR_ACUMULADO",
        "cr_periodo": "VL_CR_PERIODO",
        'codCursoIngresso': 'CD_CURSO_INGRESSO',
        'cursoIngressoUFRJ': 'DS_NOME_CURSO_INGRESSO',
        'codCursoAtual': 'CD_CURSO_ATUAL',
        'cursoAtual': 'DS_NOME_CURSO_ATUAL'
    }
)

In [181]:
f_situacao_periodo = pd.merge(
    f_situacao_periodo,
    df_d_aluno[["SK_D_ALUNO", "DS_MATRICULA_DRE"]],
    on="DS_MATRICULA_DRE",
    how="inner",
)
f_situacao_periodo = pd.merge(
    f_situacao_periodo,
    df_d_periodo[["DS_PERIODO", "SK_D_PERIODO"]],
    on="DS_PERIODO",
    how="inner",
)
f_situacao_periodo = pd.merge(
    f_situacao_periodo,
    df_d_curso[["SK_D_CURSO", "CD_CURSO_INGRESSO", "CD_CURSO_ATUAL"]],
    on=["CD_CURSO_INGRESSO", "CD_CURSO_ATUAL"],
    how="inner",
)

In [165]:
f_situacao_periodo = f_situacao_periodo[['SK_D_ALUNO', 'SK_D_PERIODO', 'SK_D_CURSO', 'VL_CR_PERIODO', 'VL_CR_ACUMULADO']]

In [183]:
f_situacao_periodo["VL_CR_PERIODO"] = pd.to_numeric(f_situacao_periodo["VL_CR_PERIODO"], errors="coerce")
f_situacao_periodo["VL_CR_ACUMULADO"] = pd.to_numeric(f_situacao_periodo["VL_CR_ACUMULADO"], errors="coerce")

In [168]:
# Salvar dados na camada Gold
carregar_dados_database(f_situacao_periodo, "f_situacao_periodo", "gold", "parquet")